### Instalation

In [ ]:
import matplotlib.pyplot as plt

def evaluating_change_point(true, prediction, metric='nab', numenta_time=None):
    """
    true - both:
                list of pandas Series with binary int labels
                pandas Series with binary int labels
    prediction - both:
                      list of pandas Series with binary int labels
                      pandas Series with binary int labels
    metric: 'nab', 'binary' (FAR, MAR), 'average_delay'
                
    """
    
    def binary(true, prediction):      
        """
        true - true binary series with 1 as anomalies
        prediction - trupredicted binary series with 1 as anomalies
        """
        def single_binary(true,prediction):
            true_ = true == 1 
            prediction_ = prediction == 1
            TP = (true_ & prediction_).sum()
            TN = (~true_ & ~prediction_).sum()
            FP = (~true_ & prediction_).sum()
            FN = (true_ & ~prediction_).sum()
            return TP,TN,FP,FN
            
        if type(true) != type(list()):
            TP,TN,FP,FN = single_binary(true,prediction)
        else:
            TP,TN,FP,FN = 0,0,0,0
            for i in range(len(true)):
                TP_,TN_,FP_,FN_ = single_binary(true[i],prediction[i])
                TP,TN,FP,FN = TP+TP_,TN+TN_,FP+FP_,FN+FN_       
    
        f1 = round(TP/(TP+(FN+FP)/2), 2)
        print(f'False Alarm Rate {round(FP/(FP+TN)*100,2)} %' )
        print(f'Missing Alarm Rate {round(FN/(FN+TP)*100,2)} %')
        print(f'F1 metric {f1}')
        return f1
    
    def average_delay(detecting_boundaries, prediction):
        
        def single_average_delay(detecting_boundaries, prediction):
            missing = 0
            detectHistory = []
            for couple in detecting_boundaries:
                t1 = couple[0]
                t2 = couple[1]
                if prediction[t1:t2].sum()==0:
                    missing+=1
                else:
                    detectHistory.append(prediction[prediction ==1][t1:t2].index[0]-t1)
            return missing, detectHistory
            
        
        if type(prediction) != type(list()):
            missing, detectHistory = single_average_delay(detecting_boundaries, prediction)
        else:
            missing, detectHistory = 0, []
            for i in range(len(prediction)):
                missing_, detectHistory_ = single_average_delay(detecting_boundaries[i], prediction[i])
                missing, detectHistory = missing+missing_, detectHistory+detectHistory_

        add = pd.Series(detectHistory).mean()
        print('Average delay', add)
        print(f'A number of missed CPs = {missing}')
        return add
    
    def evaluate_nab(detecting_boundaries, prediction, table_of_coef=None):
        """
        Scoring labeled time series by means of
        Numenta Anomaly Benchmark methodics
        Parameters
        ----------
        detecting_boundaries: list of list of two float values
            The list of lists of left and right boundary indices
            for scoring results of labeling
        prediction: pd.Series with timestamp indices, in which 1 
            is change point, and 0 in other case. 
        table_of_coef: pandas array (3x4) of float values
            Table of coefficients for NAB score function
            indeces: 'Standart','LowFP','LowFN'
            columns:'A_tp','A_fp','A_tn','A_fn'
        Returns
        -------
        Scores: numpy array, shape of 3, float
            Score for 'Standart','LowFP','LowFN' profile 
        Scores_null: numpy array, shape 3, float
            Null score for 'Standart','LowFP','LowFN' profile             
        Scores_perfect: numpy array, shape 3, float
            Perfect Score for 'Standart','LowFP','LowFN' profile  
        """
        def single_evaluate_nab(detecting_boundaries, prediction, table_of_coef=None, name_of_dataset=None):
            if table_of_coef is None:
                table_of_coef = pd.DataFrame([[1.0,-0.11,1.0,-1.0],
                                     [1.0,-0.22,1.0,-1.0],
                                      [1.0,-0.11,1.0,-2.0]])
                table_of_coef.index = ['Standart','LowFP','LowFN']
                table_of_coef.index.name = "Metric"
                table_of_coef.columns = ['A_tp','A_fp','A_tn','A_fn']

            alist = detecting_boundaries.copy()
            prediction = prediction.copy()

            Scores, Scores_perfect, Scores_null=[], [], []
            for profile in ['Standart', 'LowFP', 'LowFN']:       
                A_tp = table_of_coef['A_tp'][profile]
                A_fp = table_of_coef['A_fp'][profile]
                A_fn = table_of_coef['A_fn'][profile]
                def sigm_scale(y, A_tp, A_fp, window=1):
                    return (A_tp-A_fp)*(1/(1+np.exp(5*y/window))) + A_fp

                #First part
                score = 0
                if len(alist)>0:
                    score += prediction[:alist[0][0]].sum()*A_fp
                else:
                    score += prediction.sum()*A_fp
                #second part
                for i in range(len(alist)):
                    if i<=len(alist)-2:
                        win_space = prediction[alist[i][0]:alist[i+1][0]].copy()
                    else:
                        win_space = prediction[alist[i][0]:].copy()
                    win_fault = prediction[alist[i][0]:alist[i][1]]
                    slow_width = int(len(win_fault)/4)

                    if len(win_fault) + slow_width >= len(win_space):
                        print(f'Intersection of the windows of too wide widths for dataset {name_of_dataset}')
                        win_fault_slow = win_fault.copy()
                    else:
                        win_fault_slow= win_space[:len(win_fault)  +  slow_width]

                    win_fp = win_space[-len(win_fault_slow):]

                    if win_fault_slow.sum() == 0:
                        score+=A_fn
                    else:
                        #to get the first index
                        tr = pd.Series(win_fault_slow.values,index = range(-len(win_fault), len(win_fault_slow)-len(win_fault)))
                        tr_values= tr[tr==1].index[0]
                        tr_score = sigm_scale(tr_values, A_tp,A_fp,slow_width)
                        score += tr_score
                        score += win_fp.sum()*A_fp
                Scores.append(score)
                Scores_perfect.append(len(alist)*A_tp)
                Scores_null.append(len(alist)*A_fn)
            return np.array([np.array(Scores),np.array(Scores_null), np.array(Scores_perfect)])
       #======      
        if type(prediction) != type(list()):
            matrix = single_evaluate_nab(detecting_boundaries, prediction, table_of_coef=table_of_coef)
        else:
            matrix = np.zeros((3,3))
            for i in range(len(prediction)):
                matrix_ = single_evaluate_nab(detecting_boundaries[i], prediction[i], table_of_coef=table_of_coef,name_of_dataset=i)
                matrix = matrix + matrix_      
                
        results = {}
        desc = ['Standart', 'LowFP', 'LowFN'] 
        for t, profile_name in enumerate(desc):
            results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
            print(profile_name,' - ', results[profile_name])
        
        return results
            
            
    #=========================================================================
    if type(true) != type(list()):
        true_items = true[true==1].index
    else:
        true_items = [true[i][true[i]==1].index for i in range(len(true))]
        

    if not metric=='binary':
        def single_detecting_boundaries(true, numenta_time, true_items):
            detecting_boundaries=[]
            td = pd.Timedelta(numenta_time) if numenta_time is not None else pd.Timedelta((true.index[-1]-true.index[0])/len(true_items))  
            for val in true_items:
                detecting_boundaries.append([val, val + td])
            return detecting_boundaries
        
        if type(true) != type(list()):
            detecting_boundaries = single_detecting_boundaries(true=true, numenta_time=numenta_time, true_items=true_items)
        else:
            detecting_boundaries=[]
            for i in range(len(true)):
                detecting_boundaries.append(single_detecting_boundaries(true=true[i], numenta_time=numenta_time, true_items=true_items[i]))

    if metric== 'nab':
        return evaluate_nab(detecting_boundaries, prediction)
    elif metric=='average_delay':
        return average_delay(detecting_boundaries, prediction)
    elif metric== 'binary':
        return binary(true, prediction)
def Score_data(pred, real):
    # computing errors
    errors = np.abs(pred - real).flatten()
    # estimation
    mean = sum(errors)/len(errors)
    cov = 0
    for e in errors:
        cov += (e - mean)**2
    cov /= len(errors)

    print('mean : ', mean)
    print('cov : ', cov)
    return errors, cov, mean

# calculate Mahalanobis distance
def Mahala_distantce(x,mean,cov):
    return (x - mean)**2 / cov

def scale(A):
    return (A-np.min(A))/(np.max(A) - np.min(A))    

In [ ]:
df_server1 = pd.read_csv('/kaggle/input/benchmark-labeled-anomaly-detection-ts/server_res_eth1out_curve_61.csv')
df_server2 = pd.read_csv('/kaggle/input/benchmark-labeled-anomaly-detection-ts/rver_res_eth1out_curve_6.csv')
df_g = pd.read_csv('/kaggle/input/benchmark-labeled-anomaly-detection-ts/g.csv')
df_cpu = pd.read_csv('/kaggle/input/benchmark-labeled-anomaly-detection-ts/cpu4.csv')

In [ ]:
df_server1.head()

In [ ]:
df_server1.shape, df_server2.shape, df_g.shape, df_cpu.shape

In [ ]:
df_server1.label.value_counts(), df_server2.label.value_counts(), df_g.label.value_counts(), df_cpu.label.value_counts()

In [ ]:
df_server1.label.value_counts()/df_server1.shape[0]*100, df_server2.label.value_counts()/df_server2.shape[0]*100, df_g.label.value_counts()/df_g.shape[0]*100, df_cpu.label.value_counts()/df_cpu.shape[0]*100,

### Missing values

In [ ]:
import missingno as msno

msno.matrix(df_cpu[['timestamp', 'value']])

In [ ]:
df_cpu.head(2)

In [ ]:
# plotting the labels both for outlier and changepoint detection problems
df_cpu.value.plot(figsize=(12,3))
# df_cpu.scores.plot()
# df_cpu.scores_norm.plot()
df_cpu.outlier.plot()
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# follow lag
model_ar = ARIMA(df_cpu['value'], order=(1,1,0))  
results_ARIMA_ar = model_ar.fit(disp=-1)

In [ ]:
errors, cov, mean = Score_data(results_ARIMA_ar.fittedvalues.values.flatten(), df_cpu['value'].values.flatten()[0:len(results_ARIMA_ar.fittedvalues)])


In [ ]:
mahala_dist = []
for e in errors:
    mahala_dist.append(Mahala_distantce(e, mean, cov))

In [ ]:
df_cpu['scores'] = [0]+mahala_dist

df_cpu['scores_norm'] = scale([0]+mahala_dist)
plt.figure(figsize=(12, 8))
plt.hist(df_cpu['scores_norm'], bins=50);

In [ ]:
q1_pc1, q3_pc1 = df_cpu['scores'].quantile([0.25, 0.75])
iqr_pc1 = q3_pc1 - q1_pc1

# Calculate upper and lower bounds for outlier for pc1
lower_pc1 = q1_pc1 - (1.5*iqr_pc1)
upper_pc1 = q3_pc1 + (1.5*iqr_pc1)
    # Filter out the outliers from the pc1
df_cpu['outlier'] = ((df_cpu['scores']>upper_pc1) | (df_cpu['scores']<lower_pc1)).astype('int')

In [ ]:
true = df_cpu.label
prediction = df_cpu.outlier
score = df_cpu.scores

% anomalies

In [ ]:
df_cpu.label.value_counts() / df_cpu.shape[0]*100

# true outlier indices selection

In [ ]:
df_cpu.head(2)

In [ ]:
# true outlier indices selection

df_cpu.outlier.plot(figsize=(12,3), label='predictions', marker='o', markersize=5)

df_cpu.label.plot(marker='o', markersize=2)
plt.legend();

#### Metrics calculation

binary classification metrics calculation

In [ ]:
# binary classification metrics calculation
binary = evaluating_change_point(df_cpu.label, df_cpu.outlier, metric='binary', numenta_time='30 sec')

average detection delay metric calculation

In [ ]:
from datetime import datetime
l_datetime = []
for i in range(df_cpu.shape[0]):
    l_datetime.append(datetime.fromtimestamp(df_cpu.timestamp[i]) )

df_cpu['datetime'] =l_datetime
df_cpu.head(2)

In [ ]:
nab_df = df_cpu[['datetime','label', 'outlier']]
nab_df.index = df_cpu.datetime
# average detection delay metric calculation
add = evaluating_change_point(nab_df.label, nab_df.outlier, metric='average_delay', numenta_time='30 sec')

nab metric calculation

In [ ]:
# nab metric calculation
nab = evaluating_change_point(nab_df.label, nab_df.outlier, metric='nab', numenta_time='30 sec')

In [ ]:
def anomaly_plot(df_cpu, target, text):
    # visualization
    a = df_cpu.loc[df_cpu[target] == 1] 
    _ = plt.figure(figsize=(18,6))
    _ = plt.plot(df_cpu['value'], color='blue', label='Inline')
    _ = plt.plot(a['value'], linestyle='none', marker='X', color='red', markersize=12, label='Anomalies')
    _ = plt.xlabel('Series')
    _ = plt.ylabel('Data')
    _ = plt.title(text)
    _ = plt.legend(loc='best')
    plt.show();
    
anomaly_plot(df_cpu, 'outlier', "Predicted Anomalies")
anomaly_plot(df_cpu, 'label', "Truth Anomalies")

In [ ]:
#2 -- Distributions of Predicted Probabilities of both classes
labels=['Positivo','negativo']
plt.hist(df_cpu[df_cpu['outlier']==1]['scores_norm'], density=True, bins=25,
             alpha=.5, color='green',  label=labels[0])
plt.hist(df_cpu[df_cpu['outlier']==0]['scores_norm'], density=True, bins=25,
             alpha=.5, color='red', label=labels[1])
plt.axvline(.5, color='blue', linestyle='--', label='Fronteira de Decisão')
# plt.xlim([0,1])
plt.title('Distribuição dos Valores', size=13)
plt.xlabel('Valores normalizados', size=13)
plt.ylabel('Amostra (normalizados)', size=13)
plt.legend(loc="upper right")

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Com politica
print(classification_report(true, prediction))
confusion_matrix(true, prediction)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(prediction, true)